# 🧠 1. Setup and Installation

In [1]:
# Install required libraries
!pip install datasets transformers scikit-learn torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.5/74.5 MB 5.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.0/172.0 kB 3.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 5.9 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: sympy
    Found existing installation: sympy 1.12
    Uninstalling sympy-1.12:
      Successfully uninstalled sympy-1.12
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.2.0
    Uninstalling fsspec-2025.2.0:
      Successfully uninstalled fsspec-2025.2.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
universal-pathlib 0.2.6 requires fsspec!=2024.3.1,>=2022.1.0, but you have fsspec 2024.3.1 which is incompatible.


# 📥 2. Load the Dataset

In [2]:
from datasets import load_dataset

# Load the dataset
ds = load_dataset("Pulk17/Fake-News-Detection-dataset")

# Check the structure
print(ds)

/Users/rathan/anaconda3/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating train split: 100%|███| 30000/30000 [00:00<00:00, 41357.98 examples/s]

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'title', 'text', 'subject', 'date', 'label'],
        num_rows: 30000
    })
})


# 👀 3. Inspect the Data

In [3]:
# View some sample data
print(ds['train'][0])

# Check column names and number of samples
print(ds['train'].column_names)
print(f"Number of training samples: {len(ds['train'])}")

{'Unnamed: 0': 2619, 'title': "Ex-CIA head says Trump remarks on Russia interference 'disgraceful'", 'text': 'Former CIA director John Brennan on Friday criticized as “disgraceful” President Donald Trump’s efforts to play down U.S. intelligence agencies’ assessment that Russia meddled in the 2016 U.S. election. Trump’s administration has been dogged by investigations into allegations of Russian interference in last year’s U.S. presidential election and possible ties with his campaign team. Speaking one day before his first meeting with Russian President Vladimir Putin in Hamburg earlier this month, Trump said he suspected Russian interference in the election but that no one knows for sure. “These types of comments are just disgraceful ... and the person who said them should be ashamed of himself,” said Brennan, CIA chief under former President Barack Obama, at the Aspen Security Forum. Special Counsel Robert Mueller and several U.S. congressional committees are investigating whether Ru

# 🧹 4. Preprocess the Data

In [4]:
from transformers import AutoTokenizer

# Load the BERT tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# Tokenize the dataset
def tokenize_function(example):
    return tokenizer(example["text"], padding="max_length", truncation=True)

tokenized_datasets = ds.map(tokenize_function, batched=True)

# Remove unnecessary columns if any
tokenized_datasets = tokenized_datasets.remove_columns(['text'])

# Rename label column if needed
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")

# Set dataset format for PyTorch
tokenized_datasets.set_format("torch")

Map: 100%|███████████████████████| 30000/30000 [00:07<00:00, 3880.90 examples/s]


# 🔀 5. Split the Data

In [5]:
from datasets import DatasetDict

# Split train into train + validation (e.g., 80/20)
split = tokenized_datasets["train"].train_test_split(test_size=0.2, seed=42)

# Create a DatasetDict
tokenized_datasets = DatasetDict({
    "train": split["train"],
    "validation": split["test"]
})

print(tokenized_datasets)

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'title', 'subject', 'date', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 24000
    })
    validation: Dataset({
        features: ['Unnamed: 0', 'title', 'subject', 'date', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 6000
    })
})


# 🧩 6. Load a Pretrained Model

In [6]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# 7. Create a DataLoader

In [7]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(tokenized_datasets['train'], batch_size=16, shuffle=True)
val_dataloader = DataLoader(tokenized_datasets['validation'], batch_size=16)


# 8. Set Up Optimizer and Scheduler

In [8]:
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=5e-5)

# 9. Training Loop

In [9]:
import torch
from torch.nn import CrossEntropyLoss
from tqdm import tqdm
from torch import nn

# Setup device
device = "cuda" if torch.cuda.is_available() else "cpu"

# Simple model for demonstration
class SimpleClassifier(nn.Module):
    def __init__(self, num_labels=2):
        super().__init__()
        self.num_labels = num_labels
        self.linear = nn.Linear(768, num_labels)

    def forward(self, input_ids=None, attention_mask=None):
        batch_size = input_ids.shape[0]
        logits = self.linear(torch.randn(batch_size, 768).to(input_ids.device))
        return type('Obj', (object,), {'logits': logits})

model = SimpleClassifier().to(device)

# Optimizer and loss
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
loss_fn = CrossEntropyLoss()

# Training parameters
batch_size = 16
seq_len = 128
num_batches = 20
num_epochs = 3

# Training loop
for epoch in range(num_epochs):
    model.train()
    loop = tqdm(range(num_batches), leave=True)
    for i in loop:
        optimizer.zero_grad()

        # Inputs for demonstration
        input_ids = torch.randint(0, 1000, (batch_size, seq_len)).to(device)
        attention_mask = torch.ones(batch_size, seq_len).to(device)
        labels = torch.randint(0, 2, (batch_size,)).to(device)

        outputs = model(input_ids, attention_mask=attention_mask)
        loss = loss_fn(outputs.logits, labels)
        loss.backward()
        optimizer.step()

        loop.set_description(f'Epoch {epoch+1}')
        loop.set_postfix(loss=round(loss.item(), 4))


Epoch 3: 100%|█████████████████████| 20/20 [00:00<00:00, 944.25it/s, loss=0.716]


# 10. Evaluation

In [11]:
from sklearn.metrics import accuracy_score

# Switch model to evaluation mode
model.eval()

# Simulated validation parameters
val_batches = 5
batch_size = 16
seq_len = 128

all_preds = []
all_labels = []

with torch.no_grad():
    for _ in range(val_batches):
        input_ids = torch.randint(0, 1000, (batch_size, seq_len)).to(device)
        attention_mask = torch.ones(batch_size, seq_len).to(device)
        labels = torch.randint(0, 2, (batch_size,)).to(device)

        outputs = model(input_ids, attention_mask=attention_mask)
        preds = torch.argmax(outputs.logits, dim=-1)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Compute accuracy
accuracy = accuracy_score(all_labels, all_preds)
print(f"Validation Accuracy: {accuracy:.4f}")

Validation Accuracy: 0.4375
